# Creating Vector Database

In [1]:
import pandas as pd

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

apipath = r'H:\\My Drive\\config\\hbqa.txt'
import configparser

config = configparser.ConfigParser()
config.read(apipath)
secret_key = config['global']['OPENAI_API_KEY']
datapath = config['global']['DATA_FOLDER']
corpuspath = config['global']['CORPUS_FOLDER']

In [7]:
df_hbqa = pd.read_csv(datapath+"06-MahabharatHBQA.csv")
df_chunk = pd.read_csv(datapath+"03-Chunked_Book.csv")


In [8]:
df_hbqa.head(3)

,Ques_Id,Section_Id,Chunk_Id,Question,Ref_Answer,WordsInQues,WordsInAns
0,0,Book01_002,1,Who asked Sauti about his journey?,One of the Rishis beginning the conversation a...,6,12
1,1,Book01_002,1,What did the Rishis wish to hear from Sauti?,The Rishis wished to hear the wonderful narrat...,9,10
2,2,Book01_002,1,What did the Rishis ask Sauti to recite?,The Rishis asked Sauti to recite the sacred st...,8,32


In [5]:
df_chunk.head(5)

,Section_Id,Chunk_Id,Chunk,Chunk_Letters,Chunk_Words,Chunk_Approx_Tokens
0,Book01_002,1,THE MAHABHARATA ADI PARVA Section I\nOm! Havin...,7511,1214,1615.0
1,Book01_002,2,The Rishi Vyasa published this mass of knowled...,7551,1261,1677.0
2,Book01_002,3,"Vyasa executed the compilation of the Bharata,...",6986,1162,1545.0
3,Book01_002,5,'When I heard that Yudhishthira had been follo...,10832,1949,2592.0
4,Book01_002,6,"'Alas! Gandhari, destitute of children, grand-...",7755,1246,1657.0


In [ ]:
# !pip install -U sentence-transformers

In [6]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

c:\Users\hari_\anaconda3\envs\hbqa\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
%time
#sentences = ["This is an example sentence", "Each sentence is converted"]

ques_sentences = df_hbqa.Question.tolist()
ques_embeddings = model.encode(ques_sentences)

ans_sentences = df_hbqa.Ref_Answer.tolist()
ans_embeddings = model.encode(ans_sentences)

chunk_sentences = df_chunk.chunk.tolist()
chunk_embeddings = model.encode(chunk_sentences)



CPU times: total: 0 ns
Wall time: 0 ns


In [11]:
import torch

In [13]:
DEVICE='cpu'

In [14]:
qe= torch.tensor(ques_embeddings, dtype=torch.float).to(DEVICE)
ae= torch.tensor(ans_embeddings, dtype=torch.float).to(DEVICE)
ce= torch.tensor(chunk_embeddings, dtype=torch.float).to(DEVICE)

In [15]:
# Flatten the tensors into 1D arrays
qe_list = qe.tolist()
ae_list = ae.tolist()
ce_list = ce.tolist()

In [16]:
df_CQAembed = pd.DataFrame({ "ChunkVector" : ce_list, "QuestVector": qe_list, "AnsVector": ae_list,})
df_CQAembed.to_csv(datapath+"07-Embedding_CQA.csv")

In [17]:
df_CQAembed

,quest_embed,ans_embed,context_embed
0,"[-0.03560031205415726, 0.04083798825740814, -0...","[-0.058902308344841, 0.06546207517385483, -0.0...","[-0.03560031205415726, 0.04083798825740814, -0..."
1,"[-0.05214887112379074, 0.07548260688781738, -0...","[0.00867170188575983, 0.05012994632124901, -0....","[-0.05214887112379074, 0.07548260688781738, -0..."
2,"[-0.10114305466413498, 0.006543009541928768, -...","[-0.038608163595199585, 0.0747721791267395, -0...","[-0.10114305466413498, 0.006543009541928768, -..."
3,"[-0.0659937858581543, 0.015136297792196274, 0....","[-0.07434075325727463, -0.009906511753797531, ...","[-0.0659937858581543, 0.015136297792196274, 0...."
4,"[0.005312507972121239, 0.0136681804433465, -0....","[-0.02026970498263836, -0.00869344174861908, -...","[0.005312507972121239, 0.0136681804433465, -0...."
...,...,...,...
1114,"[-0.13002479076385498, 0.05591632053256035, 0....","[-0.11036577075719833, 0.08855369687080383, 0....","[-0.13002479076385498, 0.05591632053256035, 0...."
1115,"[0.008280224166810513, -0.0024108337238430977,...","[0.015167448669672012, 0.04057442396879196, 0....","[0.008280224166810513, -0.0024108337238430977,..."
1116,"[-0.00033917734981514513, 0.04992949590086937,...","[0.04992251843214035, 0.06294243782758713, -0....","[-0.00033917734981514513, 0.04992949590086937,..."
1117,"[0.015867557376623154, 0.026233725249767303, -...","[0.025380071252584457, 0.07433845847845078, -0...","[0.015867557376623154, 0.026233725249767303, -..."


In [ ]:
!pip install pinecone-client

In [ ]:
import pinecone      

pinecone.init(      
	api_key='ddf2ef12-9108-4e80-b438-8f86a0b617c0',      
	environment='asia-southeast1-gcp-free'      
)      
index = pinecone.Index('EXAMPLE_INDEX')

In [19]:
import os
import pinecone

# get api key from app.pinecone.io
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY') or 'PINECONE_API_KEY'
# find your environment next to the api key in pinecone console
PINECONE_ENV = os.environ.get('PINECONE_ENVIRONMENT') or 'PINECONE_ENVIRONMENT'

pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENV
)

In [21]:
index_name = 'semantic-search-fast'

In [29]:
!pip uninstall pinecone_datasets

^C


In [27]:
from pinecone_datasets import load_dataset

dataset = load_dataset('quora_all-MiniLM-L6-bm25')
# # we drop sparse_values as they are not needed for this example
# dataset.documents.drop(['metadata'], axis=1, inplace=True)
# dataset.documents.rename(columns={'blob': 'metadata'}, inplace=True)
# # we will use 80K rows of the dataset between rows 240K -> 320K
# dataset.documents.drop(dataset.documents.index[320_000:], inplace=True)
# dataset.documents.drop(dataset.documents.index[:240_000], inplace=True)
# dataset.head()

ModuleNotFoundError: No module named 'pinecone_datasets'

In [ ]:
import time

# only create index if it doesn't exist
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        name=index_name,
        dimension=len(dataset.documents.iloc[0]['values']),
        metric='cosine'
    )
    # wait a moment for the index to be fully initialized
    time.sleep(1)

# now connect to the index
index = pinecone.GRPCIndex(index_name)